In [ ]:
#Este Notebook é para ver quais são as tabelas dentro do schema Sales do AdventureWorks e depois extrair as tabelas no catalog Raw. 
user = dbutils.secrets.get("pietro_secrets_scope", "user")
password = dbutils.secrets.get("pietro_secrets_scope", "password")
port = dbutils.secrets.get("pietro_secrets_scope", "port")
ip = dbutils.secrets.get("pietro_secrets_scope", "ip")


jdbc_url = (
    f"jdbc:sqlserver://{ip}:{port};"
    "databaseName=AdventureWorks;"
    "encrypt=true;"
    "trustServerCertificate=true"
)

connection_properties = {
    "user": user,
    "password": password,
    "driver": "com.microsoft.sqlserver.jdbc.SQLServerDriver"
}

query = """
SELECT TABLE_SCHEMA, TABLE_NAME
FROM INFORMATION_SCHEMA.TABLES
WHERE TABLE_SCHEMA = 'Sales'
  AND TABLE_TYPE = 'BASE TABLE'
"""

tables_df = spark.read.jdbc(
    jdbc_url,
    f"({query}) AS tables",
    properties=connection_properties
)

tables_list = [row["TABLE_NAME"] for row in tables_df.collect()]

for table in tables_list:
    df = spark.read.jdbc(jdbc_url, f"Sales.{table}", properties=connection_properties)
    
    table_name = table  
    df.write.format("delta").mode("overwrite").saveAsTable(f"pietro_longui_raw.dev_pietro_longui_sales_raw.{table_name}")

Tabelas base do schema Sales: ['SalesTaxRate', 'PersonCreditCard', 'SalesTerritory', 'SalesTerritoryHistory', 'ShoppingCartItem', 'SpecialOffer', 'SpecialOfferProduct', 'Store', 'CountryRegionCurrency', 'CreditCard', 'Currency', 'CurrencyRate', 'Customer', 'SalesOrderDetail', 'SalesOrderHeader', 'SalesOrderHeaderSalesReason', 'SalesPerson', 'SalesPersonQuotaHistory', 'SalesReason']
